In [ ]:
import numpy as np
import scipy as sp
from pprint import pprint
from utils import preprocess, select_dim, calc_selection_threshold, score_function_i

In [ ]:
data = preprocess('dataset_diabetes/diabetic_data.csv')
N, V = data.shape

In [ ]:
data, N, V

In [ ]:
selection_threshold = calc_selection_threshold(data)

In [ ]:
C = [[1, 2, 3], [9, 39, 43, 341], [32, 21, 432]]
select_dim(data[C[0]], selection_threshold)

In [ ]:
# labeled_objects[i][j] = j-th object in i-th cluster.
labeled_objects = [[9, 23, 543], [456, 34, 52], [76, 500, 381, 245]]

# labeled_dimensions[i][j] = j-th dimension relevant to i-th cluster.
labeled_dimensions = [[1, 3, 5], [9, 7, 8], [4, 2, 6]]    

In [ ]:
def bin_cell(data, edges):
    """
    Bin samples in data into the cell defined by edges.
    """
    res = []
    for i in range(len(data)):
        sample = data[i]
        
        # Check whether sample lies in the cell.
        in_cell = True
        for j in range(len(sample)):
            if sample[j] < edges[j][0] or sample[j] > edges[j][1]:
                in_cell = False
                break
        
        # Add it to the results if so.
        if in_cell:
            res.append(i)
    return res

In [ ]:
def define_edges(centre, edge_lengths):
    """
    Define the cell edges from a centre and the edge lengths.
    """
    edges = [(centre[i] - edge_lengths[i] / 2, centre[i] + edge_lengths[i] / 2) for i in range(len(edge_lengths))]
    return edges

In [ ]:
def hill_climb(data, curr_centre, step_lengths):
    """
    Hill-climbing to find the cell with highest density. 
    """
    # Find the central cell count.
    curr_edges = define_edges(curr_centre, step_lengths)
    curr_bin = bin_cell(data, curr_edges)
    
    # Find the denser cell than current centre.
    denser_found = False
    max_centre = curr_centre
    max_bin = curr_bin
    
    # Explore the neighbouring cells.
    for i in range(len(step_lengths)):
        for sign in [-1, 1]:
            
            # Move to the neighbouring centre.
            step_length = step_lengths[i]
            cell_centre = curr_centre.copy()
            cell_centre[i] += sign * step_length

            # Bin the neighbouring cell.
            cell_edges = define_edges(cell_centre, step_lengths)
            cell_bin = bin_cell(data, cell_edges)

            # Find the most dense cell.
            if len(cell_bin) > len(max_bin):
                max_centre = cell_centre
                max_bin = cell_bin
                denser_found = True
    
    # Found the maximum.
    if not denser_found:
        return max_bin
    else:
        return hill_climb(data, max_centre, step_lengths)

In [ ]:
def get_peak(data, step_lengths):
    """
    Find the absolute density peak of the grid.
    """
    
    # Bin the data into cells.
    vals_range = np.ptp(data, axis=0)
    bin_nums = [vals_range[i] / step_lengths[i] for i in range(len(step_lengths))]
    H, edges = np.histogramdd(data, bin_nums)
    
    # Find absolute density peak.
    max_indices = np.unravel_index(H.argmax(), H.shape)
    max_edges = [(edges[i][max_indices[i]], edges[i][max_indices[i] + 1]) for i in range(len(edges))]
    
    # Find data points in the peak cell.
    return bin_cell(data, max_edges)

In [ ]:
def max_min_dist(data, sd_data, G, selected_dims):
    """
    Sort the ungrouped data points according to min-dist to grouped points.
    """
    # Find all ungrouped points.
    ungrouped_points = np.arange(len(data))
    ungrouped_points = np.delete(ungrouped_points, np.concatenate((G.values())))
    up_dist = {}
    
    # Loop over private seed groups.
    for i, G_i in G.iteritems():
        
        # Loop over private seeds.
        for j in G_i:
            
            # Loop over ungrouped points.
            for up in ungrouped_points:
                
                # Calculate the standardized distance.
                sd_dist = 0
                for dim in selected_dims[i]:
                    sd_dist += ((data[up][dim] - data[j][dim]) / sd_data[dim]) ** 2
                sd_dist = sd_dist ** 0.5
                
                # Compare it with current minimum.
                if up not in up_dist or up_dist[up] > sd_dist:
                    up_dist[up] = sd_dist
    return max(up_dist.iterkeys(), key=(lambda key: up_dist[key]))

In [ ]:
def private_seeds_for_labeled_objects(data, labeled_objects_i, labeled_dimensions_i=None):
    selection_threshold = calc_selection_threshold(data)
    
    # Extract i-th cluster's labeled objects.
    data_i = data[labeled_objects_i]
    med_i = np.median(data_i, axis=0)

    # Define a candidate set that includes the dimensions selected by SelectDim as well as 
    # those in labeled_dimensions (if any).
    candidate_set = select_dim(data_i, selection_threshold, med_i)
    if labeled_dimensions_i: 
        candidate_set = list(set(np.concatenate((candidate_set, labeled_dimensions_i))))

    # TODO: Ask Vic to fix score function.
    # score_ij = score_function_i(i, candidate_set, med_i)
    score_ij = np.random.rand(len(candidate_set))

    # Each dimension in the set has a probability proportional to 
    # the score function of being selected as a building dimension of a grid.
    score_ij /= sum(score_ij)
    building_dims = np.random.choice(a=candidate_set, size=building_dim_num, replace=False, p=score_ij)

    # Extract the data with building dimensions.
    data_grid = data[:, building_dims]
    sd_grid = np.std(data_grid, axis=0)
    med_i_grid = med_i[building_dims]

    # Apply hill-climbing search to find most dense cell.
    G_i = hill_climb(data_grid, med_i_grid, sd_grid)
    return G_i

In [ ]:
building_dim_num = 3

def initialize(data, k, labeled_objects, labeled_dimensions):
    selection_threshold = calc_selection_threshold(data)
    sd_data = np.std(data, axis=0)
    
    # i-th sample belongs to seed_group_label[i] seed group.
    seed_group_label = np.zeros(len(data))
    
    # Denote the current distance to the closest median.
    med_dist = np.zeros(len(data))
    
    # Private seeds.
    G = {}
    
    # Selected dimensions.
    selected_dims = {}
    
    # Loop over clusters with both labeled objects and labeled dimensions.
    for i in range(k):
        if ((i < len(labeled_objects) and len(labeled_objects[i]) > 0) 
            and (i < len(labeled_dimensions) and len(labeled_dimensions[i]) > 0)):
            G[i] = private_seeds_for_labeled_objects(data, labeled_objects[i], labeled_dimensions[i])
            
            # Find the relevant dimensions for this cluster.
            selected_dims[i] = select_dim(data[G[i]], selection_threshold)
    
    # Loop over clusters with only labeled objects.
    for i in range(k):
        if ((i < len(labeled_objects) and len(labeled_objects[i]) > 0) 
            and not ((i < len(labeled_dimensions) and len(labeled_dimensions[i]) > 0))):
            G[i] = private_seeds_for_labeled_objects(data, labeled_objects[i])
            
            # Find the relevant dimensions for this cluster.
            selected_dims[i] = select_dim(data[G[i]], selection_threshold)
        
    # Loop over clusters with only labeled dimensions.
    for i in range(k):
        if ((not (i < len(labeled_objects) and len(labeled_objects[i]) > 0)) 
            and (i < len(labeled_dimensions) and len(labeled_dimensions[i]) > 0)):
            candidate_set = labeled_dimensions[i]
            building_dims = np.random.choice(a=candidate_set, size=building_dim_num, replace=False)

            # Extract the data with building dimensions.
            data_grid = data[:, building_dims]
            sd_grid = np.std(data_grid, axis=0)

            # Find absolute peak.
            G[i] = get_peak(data_grid, sd_grid)
            
            # Find the relevant dimensions for this cluster.
            selected_dims[i] = select_dim(data[G[i]], selection_threshold)
    
    # Loop over clusters with no input.
    for i in range(k):
        if not ((i < len(labeled_objects) and len(labeled_objects[i]) > 0)
            and (i < len(labeled_dimensions) and len(labeled_dimensions[i]) > 0)):
            
            # Pick the seed whose minimum distance to all the seeds already picked 
            # by other seed groups is maximum, as the median.
            med_i = data[max_min_dist(data, sd_data, G, selected_dims)]
            G[i] = hill_climb(data, med_i, sd_data)
            
            # Find the relevant dimensions for this cluster.
            selected_dims[i] = select_dim(data[G[i]], selection_threshold)
    return G, selected_dims
G, selected_dims = initialize(data, 5, labeled_objects, labeled_dimensions)

In [ ]:
# Simple closest centre search.
for j in range(len(data_grid)):
    sample = data_grid[j]

    # Check whether the point lies within the grid.
    in_grid = True

    # Calculate the standardized distance to the median.
    std_dist = 0
    for building_dim in range(building_dim_num):
        std_dist_comp = abs(sample[building_dim] - med_i[building_dim]) / sd_grid[building_dim]
        if std_dist > 1:
            in_grid = False
            break
        std_dist += std_dist_comp ** 2
    std_dist = std_dist ** 0.5

    # If the point lies within the grid, check whether this is the closest median.
    if in_grid and (seed_group_label[j] == 0 or std_dist < med_dist[j]):

        # Update its label and closest to-median distance.
        if seed_group_label[j] != 0:
            G[seed_group_label[j] - 1].remove(j)
        G_i.append(j)
        seed_group_label[j] = i + 1
        med_dist[j] = std_dist

In [ ]:
r = np.random.randn(100,3)
H, edges = np.histogramdd(r, bins = (5, 8, 4))
edges